In [2]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.models.resnet as resnet

In [3]:
import visdom
vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [4]:
def value_tracker(value_plot, value ,num):
    vis.line(X=num,
             Y=value,
             win=value_plot,
             update="append")

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [12]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True, download=True, transform=transform)

print(trainset.data.shape)

train_data_mean = trainset.data.mean( axis=(0,1,2) )
train_data_std = trainset.data.std( axis=(0,1,2) )


print(train_data_mean)
print(train_data_std)

train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [13]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trainset = torchvision.datasets.CIFAR10(root="./cifar10_data", train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=0)
testset = torchvision.datasets.CIFAR10(root="./cifar10_data",train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset,batch_size=256,
                                         shuffle=True, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [14]:
conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

In [18]:
class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes=1000,zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        
        self.layer1 = self._make_layer(block, 16,layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32,layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64,layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128,layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
                    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion)
            )
            
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

In [21]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3],10,True).to(device)

In [23]:
a=torch.Tensor(1,3,32,32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward>)


In [24]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(resnet50.parameters(), lr = 0.1, momentum=0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## make plot

In [25]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title="loss_tracker",legend=["loss"],
                                                         showlegend=True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title="Accuracy", legend=["Acc"],
                                                       showlegend=True))



## define acc_check function

In [26]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            
            _, predicted = torch.max(outputs.data,1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = (100*correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save:
        torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

## Training with acc check + model save

In [ ]:
print(len(trainloader))
epochs = 150
for epoch in range(epochs):
    running_loss = 0.0
    lr_sche.step()
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    acc = acc_check(resnet50, testloader, epoch, save=1)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    
print("Training Finished")

196
[1,    30] loss: 2.077
[1,    60] loss: 1.904
[1,    90] loss: 1.746
[1,   120] loss: 1.711
[1,   150] loss: 1.625
[1,   180] loss: 1.560
Accuracy of the network on the 10000 test images: 40 %
[2,    30] loss: 1.508
[2,    60] loss: 1.448
[2,    90] loss: 1.390
[2,   120] loss: 1.348
[2,   150] loss: 1.325
[2,   180] loss: 1.232
Accuracy of the network on the 10000 test images: 49 %
[3,    30] loss: 1.172
[3,    60] loss: 1.136
[3,    90] loss: 1.104
[3,   120] loss: 1.098
[3,   150] loss: 1.047
[3,   180] loss: 1.044
Accuracy of the network on the 10000 test images: 57 %
[4,    30] loss: 0.989
[4,    60] loss: 0.965
[4,    90] loss: 0.974
[4,   120] loss: 0.939
[4,   150] loss: 0.915
[4,   180] loss: 0.925
Accuracy of the network on the 10000 test images: 61 %
[5,    30] loss: 0.887
[5,    60] loss: 0.871
[5,    90] loss: 0.837
[5,   120] loss: 0.871
[5,   150] loss: 0.830
[5,   180] loss: 0.813
Accuracy of the network on the 10000 test images: 67 %
[6,    30] loss: 0.794
[6,    6

[42,   120] loss: 0.019
[42,   150] loss: 0.017
[42,   180] loss: 0.019
Accuracy of the network on the 10000 test images: 86 %
[43,    30] loss: 0.015
[43,    60] loss: 0.016
[43,    90] loss: 0.015
[43,   120] loss: 0.015
[43,   150] loss: 0.015
[43,   180] loss: 0.018
Accuracy of the network on the 10000 test images: 86 %
[44,    30] loss: 0.015
[44,    60] loss: 0.015
[44,    90] loss: 0.016
[44,   120] loss: 0.014
[44,   150] loss: 0.017
[44,   180] loss: 0.015
Accuracy of the network on the 10000 test images: 86 %
[45,    30] loss: 0.017
[45,    60] loss: 0.015
[45,    90] loss: 0.014
[45,   120] loss: 0.016
[45,   150] loss: 0.013
[45,   180] loss: 0.017
Accuracy of the network on the 10000 test images: 85 %
[46,    30] loss: 0.016
[46,    60] loss: 0.015
[46,    90] loss: 0.016
[46,   120] loss: 0.016
[46,   150] loss: 0.016
[46,   180] loss: 0.017
Accuracy of the network on the 10000 test images: 85 %
[47,    30] loss: 0.021
[47,    60] loss: 0.016
[47,    90] loss: 0.016
[47, 

## Model Accuracy Testing

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = resnet50(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
    